In [1]:
import pandas as pd
import re
import datetime
import json
from datetime import datetime
import time
import seaborn as sb

#importorting libraries for NLP
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()


#importing librraies for data modelling 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics 
from mlxtend.evaluate import bias_variance_decomp
import xgboost as xg



profile=pd.read_csv('instagram_profiles.csv')
profile_post=pd.read_csv('instagram_posts.csv')

# Basic EDA for Instagram Profiles Data  

In [74]:
profile.shape

(537, 10)

In [76]:
profile.head()

,ig_id,username,metadata,followers,follows,total_post_count,crawled,created_at,updated_at,app_crawled
0,1181592320,randomvarun,"{""name"": ""Kumar Varun"", ""website"": ""https://yo...",152812,587,340,2023-03-28,2022-09-28 05:09:43.975459+00,2022-09-28 05:09:43.975459+00,2023-02-07 14:53:13+00
1,290516871,ig_mamba,"{""name"": ""Salman Ahmad"", ""website"": """", ""biogr...",455664,287,300,2023-03-28,2023-03-01 10:55:39.567239+00,2023-03-01 10:55:39.567239+00,NaN
2,4483453272,wakefitco,"{""name"": ""Wakefit Solutions"", ""website"": ""http...",66918,31,1592,2022-12-01,2022-12-01 10:24:18.006169+00,2022-12-01 10:24:18.006169+00,2023-01-18 18:09:23+00
3,456967256,kennethseb,"{""name"": ""Kenny Sebastian"", ""website"": ""https:...",1081986,588,1078,2023-03-28,2022-09-28 05:09:45.544014+00,2022-09-28 05:09:45.544014+00,2023-02-07 15:04:15+00
4,1173736368,danishsait,"{""name"": ""Danish sait"", ""website"": ""http://www...",1244607,697,1929,2022-09-27,2022-08-25 06:52:11.044415+00,2022-08-25 06:52:11.044415+00,2023-01-18 18:25:11+00


In [2]:
#columns with object data type
Numerical=profile.dtypes[profile.dtypes=="int64"].index
Numerical

Index(['ig_id', 'followers', 'follows', 'total_post_count'], dtype='object')

In [3]:
#columns with int data type
categorical=profile.dtypes[profile.dtypes=="object"].index
categorical

Index(['username', 'metadata', 'crawled', 'created_at', 'updated_at',
       'app_crawled'],
      dtype='object')

In [82]:
profile_copy=profile.copy()
profile_copy.drop("ig_id",axis=1,inplace=True)
profile_copy.describe(include=[np.number])

,followers,follows,total_post_count
count,5.370000e+02,537.000000,537.000000
mean,1.322691e+06,753.329609,1308.165736
std,5.074680e+06,942.460418,1315.261425
min,1.370000e+02,0.000000,0.000000
25%,7.688700e+04,152.000000,422.000000
50%,2.930930e+05,490.000000,925.000000
75%,9.495480e+05,995.000000,1715.000000
max,8.770702e+07,6364.000000,10324.000000


# Basic EDA for Instagram Profiles Data  

In [83]:
profile_post.shape

(63727, 13)

In [84]:
profile_post.head()

,ig_id,post_id,permalink_url,metadata,comments,likes,crawled,created_at,updated_at,slug,views,plays,hashtags
0,695431428,17951452121135417,https://www.instagram.com/reel/ClRAHEkJrIN/,"{""type"": ""VIDEO"", ""caption"": ""Problem of Every...",160,11359,2023-02-07,2022-11-22 14:59:24.479204+00,2022-11-22 14:59:24.479204+00,ClRAHEkJrIN,0,0,NaN
1,3114552540,17919790229430698,https://www.instagram.com/reel/Cgt-wnvBl7x/,"{""type"": ""VIDEO"", ""caption"": ""Is starbucks ove...",528,190695,2023-02-07,2023-01-13 11:29:13.752286+00,2023-01-13 11:29:13.752286+00,Cgt-wnvBl7x,2583796,4362765,"{funny,funnyvideos,starbucks,relatablememes}"
2,695431428,17906273285693178,https://www.instagram.com/reel/CmwIATFtAeq/,"{""type"": ""VIDEO"", ""caption"": ""Boys will be Con...",506,26173,2023-02-07,2022-12-29 17:29:34.072072+00,2022-12-29 17:29:34.072072+00,CmwIATFtAeq,0,0,NaN
3,210917789,18348198751004047,https://www.instagram.com/reel/CpcGEnfLeNa/,"{""type"": ""VIDEO"", ""caption"": ""All the best to ...",61,3848,2023-03-28,2023-03-06 09:56:58.910507+00,2023-03-06 09:56:58.910507+00,CpcGEnfLeNa,0,0,"{loveyouram,undalaniundhi,telugusong}"
4,52685013142,18313939324067862,https://www.instagram.com/p/Ck6FuFrp9TC/,"{""type"": ""CAROUSEL_ALBUM"", ""caption"": ""Should ...",604,164581,2023-03-28,2023-01-18 11:32:43.549401+00,2023-01-18 11:32:43.549401+00,Ck6FuFrp9TC,0,0,"{aesthetic,nature,photography,softpicture,vars..."


In [86]:
#columns with object data type
Numerical=profile_post.dtypes[profile_post.dtypes=="int64"].index
Numerical

Index(['ig_id', 'post_id', 'comments', 'likes', 'views', 'plays'], dtype='object')

In [88]:
categorical=profile_post.dtypes[profile_post.dtypes=="object"].index
categorical

Index(['permalink_url', 'metadata', 'crawled', 'created_at', 'updated_at',
       'slug', 'hashtags'],
      dtype='object')

In [90]:
profile_post_copy=profile_post.copy()
profile_post_copy.drop("post_id",axis=1,inplace=True)
profile_post_copy.describe(include=[np.number])

,ig_id,comments,likes,views,plays
count,6.372700e+04,63727.000000,6.372700e+04,6.372700e+04,6.372700e+04
mean,6.599213e+09,369.885653,6.617033e+04,1.578315e+05,2.961977e+05
std,1.282426e+10,2398.704818,1.964479e+05,7.082733e+05,1.378904e+06
min,2.246390e+05,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.786870e+08,16.000000,1.229000e+03,0.000000e+00,0.000000e+00
50%,1.797721e+09,75.000000,1.260400e+04,0.000000e+00,0.000000e+00
75%,4.795288e+09,259.000000,5.506050e+04,1.993550e+04,4.200300e+04
max,5.492189e+10,518882.000000,7.120823e+06,2.752564e+07,1.107832e+08


# Feature Engineering for Profile Data 

In [2]:
profile_post['hashtags'].fillna(0,inplace=True)
meta_list=list(profile_post['metadata'])
time_list=[]
caption_list=[]
text_caption_list=[]
hashtag_list=[]
day_of_week_list=[]
hour_list=[]
post_type_list=[]
Is_Reel_list=[]

def timestamp(i,j):
    
    global time_list
    global caption_list
    global text_caption_list
    global hashtag_list
    global day_of_week_list
    global hour_list
    global post_type_lists
    global Is_Reel_list
    
    res = json.loads(i)
    time_value=res['timestamp']
    post_type=res["type"]
    day_of_week=pd.Timestamp(time_value).dayofweek
    hour_of_day=pd.Timestamp(time_value).hour
    day_of_week_list.append(day_of_week)
    hour_list.append(hour_of_day)
    post_type_list.append(post_type)
    
    if res['hyp_type'] =='Reels':
        Is_Reel='Reels'
        Is_Reel_list.append(Is_Reel)
    else:
        Is_Reel_list.append('Non Reels')
    
    
    if 'caption' in list(res.keys()):
        caption_value=res['caption']
        text=caption_value.split('\n')[0]
        text_caption_list.append(text)
        caption_list.append(caption_value)
        
        time_list.append(time_value)
    else:
        time_list.append(time_value)
        caption_list.append('No Caption')
        text_caption_list.append('No Caption')
        
        
    if j!=0:
        no_of_hashtags=len(j.strip('{}').split(','))
        hashtag_list.append(no_of_hashtags)
    else:
        no_of_hashtags=0
        hashtag_list.append(no_of_hashtags)
        
    

profile_post.apply(lambda x: timestamp(x['metadata'],x['hashtags']),axis=1)
profile_post['timestamp']=time_list
profile_post['Caption']=caption_list
profile_post['Text_Caption']=text_caption_list
profile_post['No_of_Hashtags']=hashtag_list
profile_post['day_of_week']=day_of_week_list
profile_post['Hour']=hour_list
profile_post['post_type']=post_type_list
profile_post['Is_Reel']=Is_Reel_list

In [3]:
import numpy as np
#New Variable: Time_of_day
conditions = [
    (profile_post['Hour'] >= 0)&(profile_post['Hour']<=4),
    (profile_post['Hour'] > 4 )&( profile_post['Hour'] <= 12),
    (profile_post['Hour'] > 13 )& (profile_post['Hour'] <= 16),
    (profile_post['Hour'] > 16) & (profile_post['Hour'] <= 21),
    (profile_post['Hour'] > 21) & (profile_post['Hour'] <= 24)]

# create a list of the values we want to assign for each condition
values = ['Early Morning', 'Morning', 'Afternoon', 'Evening','Night']

# create a new column and use np.select to assign values to it using our lists as arguments
profile_post["Time_of_day"] = np.select(conditions, values)

In [4]:
#creating and merging data 
final_creater_dataframe=profile_post.merge(profile,on='ig_id')
final_creater_dataframe = final_creater_dataframe.sort_values(by=["ig_id","timestamp"])

#creating a new feature for average comments 
final_creater_dataframe['Avg_comments']=final_creater_dataframe.groupby('username')['comments'].transform('mean')

#creating a new feature for average likes
final_creater_dataframe['Avg_likes']=final_creater_dataframe.groupby('username')['likes'].transform('mean')
final_creater_dataframe['No of Videos']=final_creater_dataframe.groupby('username')['username'].transform('count')

In [5]:
Video_dataframe=final_creater_dataframe[final_creater_dataframe["post_type"]=='VIDEO']

In [6]:
Video_dataframe['Avg_views']=Video_dataframe.groupby('username')['views'].transform('mean')

C:\Users\Ipshita\AppData\Local\Temp/ipykernel_21860/2929304484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Video_dataframe['Avg_views']=Video_dataframe.groupby('username')['views'].transform('mean')


# Feature Engineering using NLP

In [7]:

#removing stop words
Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#removing puncuations
Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].str.replace('[{}]'.format(string.punctuation), '')

#lowercasing the words
Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].str.lower()

#lemmetizations 
Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v')                                                                                             
                                                          for word in x.split() ]))

#Setiment Analyis
Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].astype(str)

# compute sentiment scores (polarity) and labels
sentiment_scores = [round(TextBlob(var).sentiment.polarity, 3) for var in Video_dataframe['Text_Caption']]
sentiment_category = ["positive" if score >= 0  else "negative"  for score in sentiment_scores ]
Video_dataframe['Post_Sentiments']=sentiment_category

C:\Users\Ipshita\AppData\Local\Temp/ipykernel_21860/1803987018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
C:\Users\Ipshita\AppData\Local\Temp/ipykernel_21860/1803987018.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  Video_dataframe['Text_Caption'] = Video_dataframe['Text_Caption'].str.replace('[{}]'.format(string.punctuation), '')
C:\Users\Ipshita\AppData\Local\Temp/ipykernel_21860/1803987018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

# Image Features

In [8]:
Image_Feature=pd.read_csv('Image_feature.csv')
final=pd.merge(Video_dataframe,Image_Feature,on='username')

In [17]:
final.columns

Index(['ig_id', 'comments', 'likes', 'views', 'plays', 'No_of_Hashtags',
       'day_of_week', 'Hour', 'post_type', 'Is_Reel', 'Time_of_day',
       'username', 'followers', 'follows', 'total_post_count', 'Avg_comments',
       'Avg_likes', 'No of Videos', 'Avg_views', 'Post_Sentiments',
       'Unnamed: 0', 'brightness', 'face_detection'],
      dtype='object')

# Label Encoding

In [9]:
# Import label encoder
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
final['username']= label_encoder.fit_transform(final['username'])
final['Post_Sentiments']= label_encoder.fit_transform(final['Post_Sentiments'])
final['Time_of_day']=label_encoder.fit_transform(final['Time_of_day'])
final['post_type']=label_encoder.fit_transform(final['post_type'])
final['Is_Reel']=label_encoder.fit_transform(final['Is_Reel'])

# Dropping unecessary columns

In [10]:
final.drop(['crawled_y','app_crawled','metadata_x','metadata_y','slug','created_at_y','updated_at_y','crawled_y','hashtags','crawled_x','Caption','Text_Caption','permalink_url', 'created_at_x', 'updated_at_x', 'timestamp','post_id'],axis=1,inplace=True)

In [11]:
data_modelling_data=final[final['views']!=0]
test_predict=final[final['views']==0]

# Predict the views

In [18]:
#-------Feature selection------------------
X = data_modelling_data[['likes', 'comments','No_of_Hashtags','No of Videos',
       'day_of_week', 'username', 'Post_Sentiments','Time_of_day','followers','brightness','face_detection']]
Y=data_modelling_data['views']

# Train_Test_split

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,random_state=0)

# Scaling

In [20]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

sc_Y = StandardScaler()
y_train=y_train.values.reshape(-1,1)
sc_Y.fit_transform(y_train)
y_train=sc_Y.transform(y_train)
y_test=y_test.values.reshape(-1,1)
y_test=sc_Y.transform(y_test)

In [21]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',learning_rate=0.02,subsample=0.7, max_depth=3, n_estimators = 100, num_boost_round=100,seed = 200)
  

# Fitting the model
xgb_r.fit(x_train, y_train)
  
# Predict the model
pred = xgb_r.predict(x_test)


[15:20:32] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.



In [22]:
import numpy as np
def linear_metrics():
    r2_train= xgb_r.score(x_train, y_train)
    print('Train accuracy',r2_train)
    r2  = xgb_r.score(x_test, y_test)
    MAE = metrics.mean_absolute_error(y_test,pred)
    MSE = metrics.mean_squared_error(y_test,pred) 
    RMSE= (np.sqrt(metrics.mean_squared_error(y_test,pred)))
    adjusted_r_squared = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)
    VIF= 1/(1-r2)
    
    print('r-squared: {}'.format(r2))
    print('adjusted r square :{}'.format(adjusted_r_squared))
    print('VIF: {}'.format(VIF))
    
    print('---------------------------------------')
    print('Mean absolute error: {}'.format(MAE))
    print('Mean squared error : {}'.format(MSE))
    print('Root mean squared error: {}'.format(RMSE))
    
linear_metrics()

Train accuracy 0.7851632684329007
r-squared: 0.6969634449146126
adjusted r square :0.6967156267519526
VIF: 3.2999319165248147
---------------------------------------
Mean absolute error: 0.23445914264545084
Mean squared error : 0.3494253057015439
Root mean squared error: 0.591122073434535


In [254]:
data_modelling_data=final[final['plays']!=0]
test_predict=final[final['plays']==0]

In [255]:
#-------Feature selection------------------
X = data_modelling_data[['Avg_likes', 'No_of_Hashtags','No of Videos',
       'day_of_week', 'username', 'Post_Sentiments','Time_of_day','followers','brightness','face_detection']]
Y=data_modelling_data['plays']

In [256]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,random_state=140)

In [257]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

sc_Y = StandardScaler()
y_train=y_train.values.reshape(-1,1)
sc_Y.fit_transform(y_train)
y_train=sc_Y.transform(y_train)
y_test=y_test.values.reshape(-1,1)
y_test=sc_Y.transform(y_test)

In [258]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',learning_rate=0.02,subsample=1, max_depth=5, n_estimators = 200, num_boost_round=10,seed = 200)
  

# Fitting the model
xgb_r.fit(x_train, y_train)
  
# Predict the model
pred = xgb_r.predict(x_test)


[00:24:37] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "num_boost_round" } are not used.



In [259]:
import numpy as np
def linear_metrics():
    r2_train= xgb_r.score(x_train, y_train)
    print('Train accuracy',r2_train)
    r2  = xgb_r.score(x_test, y_test)
    MAE = metrics.mean_absolute_error(y_test,pred)
    MSE = metrics.mean_squared_error(y_test,pred) 
    RMSE= (np.sqrt(metrics.mean_squared_error(y_test,pred)))
    adjusted_r_squared = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)
    VIF= 1/(1-r2)
    
    print('r-squared: {}'.format(r2))
    print('adjusted r square :{}'.format(adjusted_r_squared))
    print('VIF: {}'.format(VIF))
    
    print('---------------------------------------')
    print('Mean absolute error: {}'.format(MAE))
    print('Mean squared error : {}'.format(MSE))
    print('Root mean squared error: {}'.format(RMSE))
    
linear_metrics()

Train accuracy 0.2851532488434573
r-squared: 0.23463046872001903
adjusted r square :0.2340542230256356
VIF: 1.3065584128069876
---------------------------------------
Mean absolute error: 0.30865928128236253
Mean squared error : 0.5709985361570181
Root mean squared error: 0.7556444508874648


In [262]:
#--------test data------------------------------
train__plays_data_set=data_modelling_data[['Avg_likes', 'No_of_Hashtags','No of Videos',
       'day_of_week', 'username', 'Post_Sentiments','Time_of_day','followers','brightness','face_detection','plays']]
train__plays_data_set.to_csv('train__plays_data_set.csv')

In [261]:
import pickle
filename = 'plays_prediction_model.sav'
pickle.dump(xgb_r, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.23463046872001903
